## ① データのアップデート

In [1]:
#ライブラリのインポート

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#　requestsメソッドを使って、サイトからHTMLの情報を抽出する。
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTy1ZbSq5OAVbEqScDw_Uf_KfLKydg0uUWBaBEPK_DXaBPCHmwv4ru9MzZHHnVHVmRVvLvOlndJ2ZNZ/pubhtml?gid=159569114&amp;single=true&amp;widget=true&amp;headers=false&amp;gid=0&amp;range=A:F%22'
res = requests.get(url)

In [3]:
#　BeautifulSoup()の設定を行う。
soup = BeautifulSoup(res.text,'html.parser')

In [4]:
# 最新弾の情報を抽出する。
poke_new = soup.find('div',id ='1490875147')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_new.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,349):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)
    
# データの結合を行う。
df_new = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_new.columns = df_new.iloc[0]
df_new = df_new.drop(0)

#価格が欠損値になっている行を削除する。
df_new = df_new.dropna(subset = ['買取'])

#買取データの日付を取得する。
import datetime 
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_new.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [5]:
#### 通常の情報を抽出する。
poke_norm = soup.find('div',id ='159569114')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_norm.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,3644):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)

# データの結合を行う。
df_norm = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_norm.columns = df_norm.iloc[0]
df_norm = df_norm.drop(0)

#価格が欠損値になっている行を削除する。
df_norm = df_norm.dropna(subset = ['買取価格'])

#買取データの日付を取得する。
import datetime
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_norm.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [6]:
#更新日のデータを完成させる。
#データを結合する。
df_today = pd.concat([df_norm,df_new],ignore_index = True)
df_today.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [7]:
#集計データを読み込む。
df_preday = pd.read_csv('cardrush_data.csv')

In [8]:
#結合のためのkeyを作る。(df_all)

df_today['key'] = df_today['カード名'].str.cat(df_today['型番'],sep = ' , ',na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['仕様'],sep = ' ,' ,na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['R'],sep = ' ,' ,na_rep='')

# print('df_todayのユニークなkeyの個数:',len(df_today['key'].unique()))
# print('df_todayのデータの列数:',len(df_today))
# print('df_todayのkeyに重複のあるデータ:\n',df_today[df_today['key'].duplicated()])

In [9]:
#結合のためのkeyを作る。(df_all)

df_preday['key'] = df_preday['カード名'].str.cat(df_preday['型番'],sep = ' , ',na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['仕様'],sep = ' ,' ,na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['R'],sep = ' ,' ,na_rep='')

# print('df_predayのユニークなkeyの個数:',len(df_preday['key'].unique()))
# print('df_predayのデータの列数:',len(df_preday))
# print('df_predayのkeyに重複のあるデータ:\n',df_preday[df_preday['key'].duplicated()])

In [10]:
#keyが被る行にtodayの数値を代入する。

df_preday[today] = 0
df_preday

for i in range(len(df_preday)):
    for j in range(len(df_today)):
        if df_preday.loc[i,'key'] == df_today.loc[j,'key']:
            df_preday.loc[i,today] = df_today.loc[j,today]
            break

In [11]:
count = 0
for i in range(len(df_today)):
    flg = 0
    for j in range(len(df_preday)):
        if df_today.loc[i,'key'] == df_preday.loc[j,'key']:
            flg = 1
            break
            
    if flg == 0:
        df_preday = pd.concat([df_preday,df_today[i:i+1]])
        df_preday.reset_index(drop = True,inplace=True)
        count = count + 1

In [13]:
#データの整理を行う。
df_update = df_preday
df_update = df_update.drop('key',axis = 1)

#データの上書きを行う。
df_update.to_csv('cardrush_data.csv',index = None)
df_update.to_csv('/Users/kanta/Desktop/Pokemon_work/3.data/1.cardrush/{}.csv'.format(today),index = None)

## ② データの分析

In [14]:
#数値をint型に変換する。
df_update = df_update.replace(',','', regex=True)

#欠損値の処理を行う。
df_update.fillna({'カード名':'-','仕様':'-','型番':'-','タイプ':'-'},inplace = True)
df_update.fillna(0,inplace = True)
df_update = pd.concat([df_update.iloc[:,:4],df_update.iloc[:,5:].astype(int)],axis = 1)

#前日との値差を計算する。
from datetime import date, timedelta
today = datetime.date.today()
yesterday = today - timedelta(days = 1) 

today = today.strftime('%y-%m-%d')
yesterday = yesterday.strftime('%y-%m-%d')

df_update['前日との値差'] = df_update[today] - df_update[yesterday]  

In [15]:
#値上がりしたカードを抽出する。
print('■ 前日からの値上がりしたカード')
up_num = len(df_update[df_update['前日との値差'] > 0])
pd.set_option('display.max_rows',up_num)
df_update.sort_values('前日との値差',ascending=False)[0:up_num]

■ 前日からの値上がりしたカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,21-01-29,21-01-30,前日との値差
1845,ピカチュウ,-,P,279/XY-P,150000,150000,150000,150000,180000,200000,20000
1893,ポンチョを着たピカチュウ,未開封,P,207/XY-P,60000,60000,60000,60000,60000,70000,10000
3493,プレイヤーズセレモニー,2020,P,127/S-P,20000,20000,22000,22000,22000,28000,6000
1861,ピカチュウ,-,P,061/SM-P,15000,15000,15000,15000,15000,20000,5000
1895,ポンチョを着たピカチュウ,未開封,P,208/XY-P,55000,55000,55000,55000,55000,60000,5000
1892,ポンチョを着たピカチュウ,-,P,207/XY-P,50000,50000,50000,50000,50000,55000,5000
282,ミミッキュ,ムンク,P,289/SM-P,75000,75000,75000,75000,75000,80000,5000
2365,ブラッキーEX,-,SR,082/078,8000,8000,8000,8000,8000,12000,4000
1986,タマタマ,-,UR,083/076,10000,10000,10000,10000,10000,13000,3000
1978,まるのみされたピカチュウ,-,P,105/S-P,11000,15000,15000,15000,15000,18000,3000


In [16]:
print('■ 前日からの値下がりが大きいカード')
down_num = len(df_update[df_update['前日との値差'] < 0])
pd.set_option('display.max_rows',down_num)
df_update.sort_values('前日との値差',ascending=True).head(down_num)

■ 前日からの値下がりが大きいカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,21-01-29,21-01-30,前日との値差
3533,ミュウツーGX,SR仕様,-,082/072,8000,8000,8000,8000,8000,0,-8000
3531,ミュウツーGX,-,RR,040/072,400,400,400,400,400,0,-400
3595,アーマーガアVMAX,-,HR,085/070,900,900,800,800,800,600,-200
3580,れんげきウーラオスVMAX,-,HR,083/070,2600,2700,2800,2800,2800,2600,-200
3532,ミュウツーGX,ノーマル仕様,-,042/131,150,150,150,150,150,0,-150
3593,アーマーガアV,-,SR,078/070,600,700,700,600,600,500,-100
3568,いちげきウーラオスV,-,RR,036/070,500,550,450,500,500,400,-100
3572,いちげきウーラオスVMAX,-,RRR,037/070,600,600,500,600,600,500,-100
3575,れんげきウーラオスV,-,RR,050/070,200,200,200,200,200,150,-50
3594,アーマーガアVMAX,-,RRR,056/070,250,250,200,200,200,150,-50


In [17]:
#新規に値段がついたカードを表示する。
print('■ 新規にデータを追記したカード')
df_update.tail(count)

■ 新規にデータを追記したカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,21-01-29,21-01-30,前日との値差
3691,カブ,-,U,067/070,0,0,0,0,0,10,10
3692,アシレーヌGX,-,RR,016/051,0,0,0,0,0,10,10
3693,アシレーヌGX,RR仕様,-,014/059,0,0,0,0,0,10,10


In [21]:
pd.set_option('display.max_rows',100)
df_update[df_update['カード名'].str.contains('スイクン')]

,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,21-01-29,21-01-30,前日との値差
141,スイクン,-,S,221/190,1700,1700,1700,1700,1800,1800,0
424,スイクン,-,P,150/S-P,30,30,30,30,30,30,0
2130,スイクンGX,-,SR,098/095,800,800,800,800,800,800,0
2131,スイクンGX,-,HR,106/095,3200,3200,3200,3200,3200,3200,0
2139,スイクンGX,-,RR,028/095,10,10,10,10,10,10,0
3133,スイクン＆エンテイLEGEND,-,☆,065/080+066/080,5000,5000,5000,5000,5000,5000,0
3135,ライコウ＆スイクンLEGEND,-,☆,067/080+068/080,5000,5000,5000,5000,5000,5000,0
3191,スイクン LV.33,-,R,旧裏,150,150,150,150,150,150,0
3288,スイクン,-,P,063/L-P,65000,65000,65000,65000,65000,65000,0
3361,スイクン☆,1ED,☆,032/106,75000,75000,75000,75000,75000,75000,0
